In [100]:
!pip install oracledb

In [101]:
import oracledb
import json
import random
from datetime import datetime

In [102]:
# ------------------- Conexão com o banco -------------------

In [103]:
def get_conn():
    with open("secret.txt", "r", encoding="utf-8") as f:
        creds = json.load(f)
    return oracledb.connect(
        user=creds["user"], 
        password=creds["password"], 
        dsn=creds["dsn"]
        )

In [104]:
# ------------------- Login e Cadastro -------------------

In [105]:
def register_user():
    conn = get_conn()
    cur = conn.cursor()

    print("\n=== CADASTRO ===")
    nome = input("Nome completo: ").strip()
    email = input("Email: ").strip()
    cpf = input("CPF: ").strip()
    senha = input("Senha: ").strip()

    cur.execute("""
        INSERT INTO users (name, email, cpf, password_hash, plan_id, role)
        VALUES (:1, :2, :3, :4, 1, 'user')
    """, (nome, email, cpf, senha))

    conn.commit()
    print("\n✅ Usuário cadastrado com sucesso (Plano Bronze).")

    cur.close()
    conn.close()

In [ ]:
def login_user():
    conn = conectar()
    cur = conn.cursor()

    print("\n=== LOGIN ===")
    email = input("Email: ").strip()
    senha = input("Senha: ").strip()

    cur.execute("""
        SELECT 
            u.id,
            u.name,
            u.role,
            u.plan_id,
            u.selected_challenge_id,
            p.name AS plan_name,
            p.description AS plan_description,
            p.has_ai_chat,
            p.has_specialist_chat
        FROM users u
        JOIN plans p ON u.plan_id = p.id
        WHERE u.email = :1 AND u.password_hash = :2
    """, (email, senha))

    row = cur.fetchone()

    if row is None:
        print("❌ Email ou senha incorretos.")
        cur.close()
        conn.close()
        return None

    user = {
        "id": row[0],
        "name": row[1],
        "role": row[2],
        "plan_id": row[3],
        "selected_challenge": row[4],
        "plan_name": row[5],
        "plan_description": row[6],
        "ai_chat": bool(row[7]),
        "specialist_chat": bool(row[8])
    }

    cur.close()
    conn.close()

    print(f"\n✅ Login bem-sucedido! Bem-vindo, {user['name']}!")
    print(f"Seu plano: {user['plan_name']}")
    return user


In [107]:
# MENU USUÁRIO

In [108]:
def user_menu(user):
    user_id = user["id"]
    selected_challenge = None

    while True:
        print("\n=== MENU DO USUÁRIO ===")
        print(f"Usuário: {user['name']} | Plano: {user['plan_name'].capitalize()}")
        print("1 - Escolher desafio")
        print("2 - Jogar desafio selecionado")
        print("3 - Ver meu progresso")
        if user["ai_chat"]:
            print("4 - Chat IA (tira dúvidas)")
        if user["specialist_chat"]:
            print("5 - Chat com Especialista (soft skills)")
        print("0 - Logout")
        op = input("Escolha: ").strip()

        if op == "1":
            selected_challenge = choose_challenge(user_id)
        elif op == "2":
            if selected_challenge:
                play_selected_challenge(user_id, selected_challenge)
            else:
                print("⚠️ Nenhum desafio selecionado! Escolha um antes de jogar.")
        elif op == "3":
            view_progress(user_id)
        elif op == "4" and user["ai_chat"]:
            ia_chat_simulation()
        elif op == "5" and user["specialist_chat"]:
            specialist_chat_simulation()
        elif op == "0":
            print("🔒 Logout efetuado.")
            break
        else:
            print("⚠️ Opção inválida!")

In [109]:
# ESCOLHER DESAFIO

In [110]:
def choose_challenge(user_id):
    conn = get_conn()
    cur = conn.cursor()
    cur.execute("SELECT id, title FROM challenges ORDER BY id")
    desafios = cur.fetchall()

    if not desafios:
        print("⚠️ Nenhum desafio disponível.")
        cur.close()
        conn.close()
        return None

    print("\n=== DESAFIOS DISPONÍVEIS ===")
    for cid, title in desafios:
        print(f"[{cid}] {title}")

    escolha = input("Digite o ID do desafio que deseja escolher: ").strip()
    if not escolha.isdigit() or int(escolha) not in [d[0] for d in desafios]:
        print("⚠️ ID inválido!")
        cur.close()
        conn.close()
        return None

    cid_int = int(escolha)
    cur.execute("UPDATE users SET selected_challenge_id = :1 WHERE id = :2", (cid_int, user_id))
    conn.commit()
    cur.close()
    conn.close()

    print(f"✅ Desafio {cid_int} selecionado com sucesso!")
    return cid_int

In [111]:
# MINI JOGOS

In [112]:
def desafio_quiz():
    perguntas = [
        ("O que é uma soft skill?", "habilidade comportamental"),
        ("O que é importante para o trabalho em equipe?", "comunicação"),
        ("O que é empatia?", "entender o outro")
    ]
    acertos = 0
    for p, resp in perguntas:
        r = input(f"{p} ").lower().strip()
        if r == resp:
            acertos += 1
    score = int((acertos / len(perguntas)) * 100)
    print(f"✅ Você acertou {acertos}/{len(perguntas)}! Pontuação: {score}%")
    return score

In [113]:
def desafio_adivinhacao():
    numero = random.randint(1, 10)
    tentativas = 0
    print("🎯 Adivinhe o número entre 1 e 10!")
    while True:
        try:
            palpite = int(input("Seu palpite: "))
        except ValueError:
            print("Digite um número válido.")
            continue
        tentativas += 1
        if palpite == numero:
            print(f"✅ Acertou em {tentativas} tentativas!")
            return max(100 - tentativas * 10, 0)
        elif palpite < numero:
            print("🔼 Maior!")
        else:
            print("🔽 Menor!")


In [114]:
def desafio_digitar():
    frase = "Aprender é evoluir constantemente"
    print(f"Digite a seguinte frase:\n> {frase}")
    entrada = input("Digite: ").strip()
    if entrada == frase:
        print("✅ Perfeito!")
        return 100
    else:
        print("❌ Erro na digitação!")
        return 50

In [115]:
# JOGAR DESAFIO SELECIONADO

In [116]:
def play_selected_challenge(user_id, challenge_id):
    print(f"\n🎯 Jogando desafio {challenge_id}...\n")

    if challenge_id == 1:
        score = desafio_quiz()
    elif challenge_id == 2:
        score = desafio_adivinhacao()
    elif challenge_id == 3:
        score = desafio_digitar()
    else:
        print("⚠️ Esse desafio ainda não tem jogo implementado.")
        return


    update_progress(user_id, challenge_id, score)

In [117]:
# PROGRESSO

In [118]:
def update_progress(user_id, challenge_id, score):
    conn = get_conn()
    cur = conn.cursor()
    cur.execute("""
        MERGE INTO progress p
        USING dual
        ON (p.user_id = :1 AND p.challenge_id = :2)
        WHEN MATCHED THEN
          UPDATE SET p.score = :3, p.last_update = CURRENT_TIMESTAMP
        WHEN NOT MATCHED THEN
          INSERT (user_id, challenge_id, score) VALUES (:1, :2, :3)
    """, (user_id, challenge_id, score))
    conn.commit()
    cur.close()
    conn.close()
    print("📈 Progresso atualizado com sucesso!")

In [119]:
def view_progress(user_id):
    conn = get_conn()
    cur = conn.cursor()
    cur.execute("""
        SELECT c.title, p.score, p.last_update
        FROM progress p
        JOIN challenges c ON c.id = p.challenge_id
        WHERE p.user_id = :1
        ORDER BY c.id
    """, (user_id,))
    rows = cur.fetchall()
    cur.close()
    conn.close()

    if not rows:
        print("⚠️ Nenhum progresso encontrado.")
    else:
        print("\n=== SEU PROGRESSO ===")
        for t, s, d in rows:
            print(f"- {t}: {s}% ({d})")

In [120]:
#   CHATS SIMULADOS

In [121]:
def ia_chat_simulation():
    print("\n🤖 === CHAT COM IA (simulado) ===")
    print("Digite 'sair' para encerrar.")
    while True:
        q = input("Você: ").strip()
        if q.lower() == "sair":
            break
        
        print("IA: Dica rápida — divida o problema em partes menores e aplique um exemplo prático.")

In [122]:
def specialist_chat_simulation():
    print("\n🎓 === CHAT COM ESPECIALISTA (Soft Skills) - simulado ===")
    print("Digite 'sair' para encerrar.")
    while True:
        q = input("Você: ").strip()
        if q.lower() == "sair":
            break
        print("Especialista: Experimente pedir feedback concreto e praticar por 10 minutos por dia.")

In [123]:
# MENU ADMIN

In [124]:
def admin_menu(user):
    while True:
        print("\n=== MENU ADMIN ===")
        print("1 - Criar novo desafio")
        print("2 - Ver todos os desafios")
        print("3 - Ver progresso de todos os usuários")
        print("4 - Exportar progresso (JSON)")
        print("5 - Gerenciar usuários (promover / mudar plano)")
        print("0 - Sair")
        op = input("Escolha: ").strip()

        if op == "1":
            create_challenge()
        elif op == "2":
            list_challenges()
        elif op == "3":
            view_all_progress()
        elif op == "4":
            export_progress_json()
        elif op == "5":
            manage_users()
        elif op == "0":
            break
        else:
            print("⚠️ Opção inválida!")

In [125]:
def create_challenge():
    conn = get_conn()
    cur = conn.cursor()
    title = input("Título do desafio: ").strip()
    desc = input("Descrição: ").strip()
    cur.execute("INSERT INTO challenges (title, description) VALUES (:1, :2)", (title, desc))
    conn.commit()
    cur.close()
    conn.close()
    print("✅ Desafio criado com sucesso!")

In [126]:
def list_challenges():
    conn = get_conn()
    cur = conn.cursor()
    cur.execute("SELECT id, title FROM challenges ORDER BY id")
    rows = cur.fetchall()
    cur.close()
    conn.close()
    print("\n=== DESAFIOS ===")
    for r in rows:
        print(f"[{r[0]}] {r[1]}")

In [127]:
def view_all_progress():
    conn = get_conn()
    cur = conn.cursor()
    cur.execute("""
        SELECT u.name, c.title, p.score 
        FROM progress p
        JOIN users u ON u.id = p.user_id
        JOIN challenges c ON c.id = p.challenge_id
        ORDER BY u.name, c.id
    """)
    rows = cur.fetchall()
    cur.close()
    conn.close()
    print("\n=== PROGRESSO DE TODOS ===")
    for n, t, s in rows:
        print(f"{n} - {t}: {s}%")

In [128]:
def export_progress_json():
    conn = get_conn()
    cur = conn.cursor()
    cur.execute("""
        SELECT u.name as user_name, c.title as challenge_title, p.score, p.last_update
        FROM progress p
        JOIN users u ON u.id = p.user_id
        JOIN challenges c ON c.id = p.challenge_id
        ORDER BY u.name
    """)
    data = [dict(zip([d[0] for d in cur.description], row)) for row in cur.fetchall()]
    cur.close()
    conn.close()
    with open("export_progress.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2, default=str)
    print("✅ Dados exportados para export_progress.json")

In [129]:
def manage_users():
    conn = get_conn()
    cur = conn.cursor()
    cur.execute("""
        SELECT u.id, u.name, u.email, u.role, p.id, p.name
        FROM users u JOIN plans p ON u.plan_id = p.id
        ORDER BY u.id
    """)
    users = cur.fetchall()
    print("\n=== USUÁRIOS CADASTRADOS ===")
    for u in users:
        print(f"{u[0]} - {u[1]} | {u[2]} | role: {u[3]} | plano: [{u[4]}] {u[5]}")
    uid = input("Digite o ID do usuário para gerenciar (ou Enter para voltar): ").strip()
    if not uid:
        cur.close()
        conn.close()
        return
    if not uid.isdigit():
        print("ID inválido.")
        cur.close()
        conn.close()
        return
    uid_int = int(uid)
    action = input("Escolha: [p]romover admin, [c]ambiar plano, [d]eletar usuário, [v]oltar: ").strip().lower()
    if action == "p":
        cur.execute("UPDATE users SET role='admin' WHERE id=:1", (uid_int,))
        conn.commit()
        print("✅ Usuário promovido a admin.")
    elif action == "c":
        # lista planos
        cur.execute("SELECT id, name FROM plans ORDER BY id")
        plans = cur.fetchall()
        print("Planos disponíveis:")
        for pid, pname in plans:
            print(f"{pid} - {pname}")
        pid_choice = input("Digite o ID do plano para atribuir: ").strip()
        if pid_choice.isdigit() and int(pid_choice) in [p[0] for p in plans]:
            cur.execute("UPDATE users SET plan_id = :1 WHERE id = :2", (int(pid_choice), uid_int))
            conn.commit()
            print("✅ Plano atualizado.")
        else:
            print("Plano inválido.")
    elif action == "d":
        confirm = input("Confirma exclusão do usuário? (s/n): ").strip().lower()
        if confirm == "s":
            cur.execute("DELETE FROM users WHERE id = :1", (uid_int,))
            conn.commit()
            print("🗑️ Usuário excluído.")
    else:
        print("Voltando...")
    cur.close()
    conn.close()

In [130]:
# MAIN

In [131]:
def main():
    while True:
        print("\n=== SkillUp - Aprendizado Gamificado ===")
        print("1 - Registrar")
        print("2 - Login")
        print("0 - Sair")
        op = input("Escolha: ").strip()

        if op == "1":
            register_user()
        elif op == "2":
            user = login_user()
            if user:
                if user["role"].lower() == "admin":
                    admin_menu(user)
                else:
                    user_menu(user)
        elif op == "0":
            print("Até logo!")
            break
        else:
            print("⚠️ Opção inválida!")

In [132]:
if __name__ == "__main__":
    main()


=== SkillUp - Aprendizado Gamificado ===
1 - Registrar
2 - Login
0 - Sair

--- LOGIN ---

✔ Bem-vindo(a), João Pedro Pereira Camilo! Plano: Bronze

=== MENU DO USUÁRIO ===
Usuário: João Pedro Pereira Camilo | Plano: Bronze
1 - Escolher desafio
2 - Jogar desafio selecionado
3 - Ver meu progresso


KeyError: 'ai_chat'